#Chapter10.[기본] 소비 데이터 분석과 승객 생존율 예측

#Section 01. [판다스 활용] 카드 소비 데이터 분석 ##1. 원본데이터 소개 ##2. 데이터 전처리
### 작업용 파일 생성하기

In [3]:
from google.colab import drive
#Google Colab에서 Google Drive를 사용하기 위해 drive 모듈을 불러옵니다.
drive.mount('/content/dirve')
#Google Drive를 Colab의 /content/drive 디렉터리에 마운트하여 파일을 읽고 쓸 수 있도록 합니다.

ModuleNotFoundError: No module named 'google.colab'

In [4]:
import pandas as pd
#pandas 라이브러리를 가져옵니다. 
with open('bc_card.txt', mode='rb') as f:
    #bc_card.txt 파일을 **바이너리 모드(rb)**로 엽니다.
    with open('bc_card_output.txt', mode='w') as w:
        #bc_card_output.txt 파일을 **쓰기 모드(w)**로 엽니다.
        data=f.read()
        #bc_card.txt 파일의 내용을 바이트(bytes) 형태로 읽어옵니다.
        print(type(data))
        #data의 타입을 출력
        data = data.decode('cp949')
        #cp949(한국어 인코딩)로 디코딩하여 문자열 변환합니다.
        print(type(data))
        #data의 타입 출력
        w.write(data)
        # 변환된 문자열 데이터를 bc_card_output.txt 파일에 씁니다. 

<class 'bytes'>
<class 'str'>


In [5]:
with open('bc_card_output.txt', mode='r') as f:
    #파일을 읽기 모드(r)로 열어 'bc_card_output.txt'데이터를 읽을 준비를 합니다.
    data= f.read()
    #bc_card_output.txt 파일의 내용을 바이트(bytes) 형태로 읽어옵니다.
len(data)
#data의 길이 

10235713

In [6]:
data[0:100]
#data의 첫100글자를 슬라이싱하여 가져온다.

'REG_YYMM\tMEGA_CTY_NO\tMEGA_CTY_NM\tCTY_RGN_NO\tCTY_RGN_NM\tADMI_CTY_NO\tADMI_CTY_NM\tMAIN_BUZ_CODE\tMAIN_BU'

In [7]:
split_data = data.split('\t')
#data를 \t 기준으로 나눈 리스트 생성
split_data[0:25]
# 탭으로 구분된 25개의 요소를 가져온다.

['REG_YYMM',
 'MEGA_CTY_NO',
 'MEGA_CTY_NM',
 'CTY_RGN_NO',
 'CTY_RGN_NM',
 'ADMI_CTY_NO',
 'ADMI_CTY_NM',
 'MAIN_BUZ_CODE',
 'MAIN_BUZ_DESC',
 'TP_GRP_NO',
 'TP_GRP_NM',
 'TP_BUZ_NO',
 'TP_BUZ_NM',
 'CSTMR_GUBUN',
 'CSTMR_MEGA_CTY_NO',
 'CSTMR_MEGA_CTY_NM',
 'CSTMR_CTY_RGN_NO',
 'CSTMR_CTY_RGN_NM',
 'SEX_CTGO_CD',
 'AGE_VAL',
 'FLC',
 'AMT',
 'CNT201906',
 '11',
 '서울특별시']

In [8]:
No_Column=23
# No_Column 변수에 23 정의
len(split_data)%No_Column
# split_data의 개수를 23으로 나눈 나머지를 계산.

3

In [9]:
type(data),type(split_data)
#data,split_data의 타입 확인

(str, list)

In [10]:
data.find('CNT')
#data의 'cnt' 가처음등장하는 인덱스 반환

257

In [11]:
data=data[:260]+'\t'+data[260:]
# 260, 261인덱스 사이에 \t를 넣는다.
split_data=data.split('\t')
#\t를 기준으로 리스트형으로 변환한다.
split_data[22],split_data[23]
# 22번째와 23번째 출력

('CNT', '201906')

In [12]:
len(split_data) % No_Column
#split_data의 길이를 23으로 나눈 나머지

4

In [13]:
split_data[23],split_data[23+22]
#23번째요소 , 45번째 요소

('201906', '1892201906')

In [14]:
split_data[23+22].find('201906')
#45번요소의 201906이 포함된 인덱스를 찾고 찾으면 해당 위치의 인덱스를 반환, 없으면 -1 반환. 

4

In [15]:
Cycle=22
# Cycle 변수에 22 저장
i=1 # 첫 번째,두 번쨰 고객 레코드 사이에 cnt와 연도를 구분하기위해 탭(\t)을 넣으므로
#루프 초기값
while i < round((len(split_data)-Cycle)/Cycle):
    #i 가 splitdata에 22를 빼고 나눈 나머지의 반올림만큼 반복한다.
    target_index = split_data[No_Column+Cycle*i].find('201906')
    # 타겟 인덱스를 split_data의 22개의 단위로 반복되는 데이터에서 '201906'이 포함된 필드 선택
    split_data[No_Column+Cycle*i]=split_data[No_Column+Cycle*i][:target_index]+'\t'+split_data[No_Column+Cycle*i][target_index:]
    # 찾은 타겟위치에 탭 추가하여 나눠둔다.
    i+=1
    #반복

In [16]:
split_data[23],split_data[23+22]
# 23번째 요소와 , 45번째 요소 재확인 (\t들어갔는지)

('201906', '1892\t201906')

In [17]:
a= ['a','b','c','d','1','2','3']
# a변수를 리스트 및 문자열 까지 정의
print(a) # a 출력
print() # 줄바꿈
result1="*****".join(a) # join함수로 리스트를 "*****"로 구분지음
print(result1) # 변환된 문자열 출력

['a', 'b', 'c', 'd', '1', '2', '3']

a*****b*****c*****d*****1*****2*****3


In [19]:
join_split_data='\t'.join(split_data)
# join 함수를 사용하여 split_data리스트의 요소를 \t로 구분지어 나눔 
join_split_data[:24] 
# 변환된함수의 문자열을 0부터23번가지 24개 출력.

'REG_YYMM\tMEGA_CTY_NO\tMEG'

In [21]:
joined_splited_data= join_split_data.split('\t')
#joined_splited_data에 join_split_data의 '\t'를 기준으로 나누어 저장함
joined_splited_data[0:23]
# 해당 리스트를 0부터22번까지 23개까지 출력함 

['REG_YYMM',
 'MEGA_CTY_NO',
 'MEGA_CTY_NM',
 'CTY_RGN_NO',
 'CTY_RGN_NM',
 'ADMI_CTY_NO',
 'ADMI_CTY_NM',
 'MAIN_BUZ_CODE',
 'MAIN_BUZ_DESC',
 'TP_GRP_NO',
 'TP_GRP_NM',
 'TP_BUZ_NO',
 'TP_BUZ_NM',
 'CSTMR_GUBUN',
 'CSTMR_MEGA_CTY_NO',
 'CSTMR_MEGA_CTY_NM',
 'CSTMR_CTY_RGN_NO',
 'CSTMR_CTY_RGN_NM',
 'SEX_CTGO_CD',
 'AGE_VAL',
 'FLC',
 'AMT',
 'CNT']

In [27]:
total=[joined_splited_data[i:i+No_Column] for i in range(0,len(joined_splited_data),No_Column)]
# 0부터 끝까지 No_Column씩 건너뛰며 반복하고 한 번에 No_Column개씩 잘라서 하위 리스트로 만들기
total[1]
# 1번 인덱스의 값 출력

['201906',
 '11',
 '서울특별시',
 '1162',
 '관악구',
 '11620585',
 '낙성대동',
 '80',
 '음식',
 '80',
 '일반음식',
 '8006',
 '서양음식',
 '내국인',
 '11',
 '서울특별시',
 '1162',
 '관악구',
 '2',
 '30대',
 '2',
 '26284804',
 '1892']

### 데이터프레임 형식으로 저장하기

In [29]:
total=[joined_splited_data[i:i+No_Column] for i in range(0,len(joined_splited_data),No_Column)]
total[-1]
#해당리스트의 마지막 요소 호출

['201906',
 '11',
 '서울특별시',
 '1135',
 '노원구',
 '11350710',
 '상계9동',
 '20',
 '문화',
 '51',
 '학원',
 '5121',
 '초중고교육기관',
 '내국인',
 '41',
 '경기도',
 '4115',
 '의정부시',
 '1',
 '40대',
 '3',
 '5227500',
 '7']

In [30]:
len(total)

100002

In [31]:
import pandas as pd #pandas 라이브러리 임포트
total_df=pd.DataFrame(total[1:],columns=total[0]) 
# total_df 를 pandas 라이브러리를 활용하여, frame을 잡는다.
#### 첫 번째 행(total[0])을 컬럼명으로, 나머지 행을 데이터로 사용해 DataFrame 생성
total_df
# 출력해보기

,REG_YYMM,MEGA_CTY_NO,MEGA_CTY_NM,CTY_RGN_NO,CTY_RGN_NM,ADMI_CTY_NO,ADMI_CTY_NM,MAIN_BUZ_CODE,MAIN_BUZ_DESC,TP_GRP_NO,...,CSTMR_GUBUN,CSTMR_MEGA_CTY_NO,CSTMR_MEGA_CTY_NM,CSTMR_CTY_RGN_NO,CSTMR_CTY_RGN_NM,SEX_CTGO_CD,AGE_VAL,FLC,AMT,CNT
0,201906,11,서울특별시,1162,관악구,11620585,낙성대동,80,음식,80,...,내국인,11,서울특별시,1162,관악구,2,30대,2,26284804,1892
1,201906,11,서울특별시,1159,동작구,11590560,상도4동,30,생활,40,...,내국인,11,서울특별시,1165,서초구,2,20대,1,109290,18
2,201906,11,서울특별시,1162,관악구,11620595,청룡동,30,생활,83,...,내국인,11,서울특별시,1162,관악구,1,20대,1,268850,52
3,201906,11,서울특별시,1144,마포구,11440660,서교동,80,음식,80,...,내국인,11,서울특별시,1138,은평구,1,20대,1,44174450,1790
4,201906,11,서울특별시,1120,성동구,11200550,사근동,80,음식,80,...,내국인,11,서울특별시,1120,성동구,1,20대,1,60338146,3536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,201906,11,서울특별시,1165,서초구,11650520,서초2동,30,생활,40,...,내국인,43,충청북도,4311,청주시,1,50대,4,50600,10
99997,201906,11,서울특별시,1117,용산구,11170520,용산2가동,30,생활,40,...,내국인,11,서울특별시,1162,관악구,1,40대,2,38640,7
99998,201906,11,서울특별시,1156,영등포구,11560535,영등포동,30,생활,40,...,내국인,28,인천광역시,2817,미추홀구,2,30대,2,340590,15
99999,201906,11,서울특별시,1141,서대문구,11410585,신촌동,40,쇼핑,42,...,내국인,44,충청남도,4413,천안시,1,20대,2,117100,3
